## Аггрегация разметки датасета ruWorldTree

Аггрегация строится по следующей системе:

1. Сбор размеченных пулов с Толоки. Возможны варианты:
    - только общий пул нужно аггрегировать, тогда забирается только он
    - часть данных находится в контрольных заданиях и экзамене, тогда к основному пулу добавляются данные задания
2. Фильтрация разметчиков:
    - в общем пуле есть некоторое количество заранее размеченных заданий - контрольных
    - хорошим считается разметчик, который показывает `accuracy >= 0.5` на данных заданиях
    - формируется список "плохих" разметчиков
3. Аггрегация ответов разметчиков по заданиям:
    - форматирование в заданиях может отличаться от изначального из-за выгрузки с Толоки
    - учитываются только ответы "хороших" разметчиков
    - аггрегация по подготовленным пулам - создается массив карточек вида {key: value}, где key - кортеж из всех значимых элементов задания, value - список из кортежей вида (user_id, answer)
4. Голосование большинством по каждому заданию:
    - минимально необходимое большинство составляет 3 голоса, так как такое большинство валидно для перекрытия 5
    - по результату формируется датафрейм с заданиями и ответами
5. Подгрузка оригинальных данных с разметкой в виде таблицы с заданиями и ответами
6. Соединение таблиц:
    - очистка форматирования в таблице с ответами разметчиков и в таблице с правильными ответами
    - создание единых столбцов с полным заданием
    - соединение таблиц по данному столбцу
    - валидация размеров
7. Подсчет метрик

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.metrics import f1_score

### Сбор данных разметки и фильтрация разметчиков

Датасет для разметки состоит из 525 объектов.

In [2]:
assignments = pd.read_csv('assignments_from_pool_42366885__28-11-2023.tsv', sep='\t')
skills = pd.read_csv('workerSkills.csv', sep='|')

Разметчикам предлагалось на основании контекста из решенных пяти примеров и одного нерешенного примера ответить на вопрос, чему равен нерешенный пример, если заменить в нем специальный символ `->` соответственно контексту.
Вход: 
- INPUT:question (пример: `Правда, что Солнце вращается вокруг Земли?`).
- INPUT:option_a (пример: `Правда`).
- INPUT:option_b (пример: `Неправда`).
- INPUT:option_c (пример: `Недоказуемо`).
- INPUT:option_d (пример: `Суждение логически противоречиво`).

Выход:
- OUTPUT:answer (одна из четырех букв: `A`, `B`, `C`, `D`,).

In [3]:
assignments.head(1)

,INPUT:option_a,INPUT:option_b,INPUT:option_c,INPUT:option_d,INPUT:question,OUTPUT:answer,GOLDEN:answer,HINT:text,HINT:default_language,ASSIGNMENT:link,ASSIGNMENT:task_id,ASSIGNMENT:assignment_id,ASSIGNMENT:task_suite_id,ASSIGNMENT:worker_id,ASSIGNMENT:status,ASSIGNMENT:started,ASSIGNMENT:submitted,ASSIGNMENT:accepted,ASSIGNMENT:reward
0,Дождь,Лед,Мороз,Пар,Что из этого является примером жидкой воды?,A,A,NaN,NaN,https://platform.toloka.ai/task/42366885/00028...,00028677a5--656635d172d52b5a0213fee2,00028677a5--65663c5aedc5b13fcc9bec79,00028677a5--65663c59edc5b13fcc9bec77,6853949dd8cb10f36cf4cade63a12f82,APPROVED,2023-11-28T19:15:38.042,2023-11-28T19:16:13.892,2023-11-28T19:16:13.892,0.03


Фильтруем толокеров с `accuracy < 0.5` на контрольных заданиях, чтобы не учитывать их ответы при подсчете метрик.

In [4]:
from collections import defaultdict

users_dict = defaultdict(lambda: defaultdict(int))

for idx, row in assignments.iterrows():
    question = row[4]

    out = row[5]
    
    gold = row[6]

    user = row[13]

    if str(user) != "nan" and str(gold) != "nan":
        if out == gold:
            users_dict[user]["good"] += 1
        else:
            users_dict[user]["bad"] += 1

print("Users total: ", len(users_dict))
bad_users = []
for key, value in users_dict.items():
    percentage_good = value["good"]/(value["good"] + value["bad"])
    if percentage_good < 0.5:
        bad_users.append(key)

print("Bad users:", len(bad_users))

Users total:  191
Bad users: 41


41 из 191 разметчиков на контрольных заданиях показали слишком плохое качество, чтобы учитывать их ответы для расчета метрики.

Отделяем контроль от основы, так как контрольные задания создавались отдельно и не должны учитываться при подсчете метрик. На контрольных заданиях есть `GOLDEN:answer`. Также отсеиваем возможные баги Толоки, когда в строке может не быть задания - `INPUT:question` содержит NaN.

In [5]:
assignments_no_control = assignments[assignments['GOLDEN:answer'].isnull()]
assignments_no_control_no_null = assignments_no_control[assignments_no_control['INPUT:question'].notnull()]

Посчитаем, сколько было затрачено на получение разметки тестовых данных без учета контрольных заданий, так как они могли проходиться неограниченное количество раз одним и тем же разметчиком.

In [6]:
def w_sum(df):
    idx = df.index.values
    vals = df.values
    summ = idx * vals
    return summ.sum()

d1 = assignments_no_control_no_null['ASSIGNMENT:reward'].value_counts(normalize=True)
d2 = assignments_no_control_no_null['ASSIGNMENT:reward'].value_counts()
print(f'взвешенная цена айтема в тесте: {round(w_sum(d1), 3)}')
print(f'потрачено на разметку теста: {round(w_sum(d2), 3)}')
print(f'{round(w_sum(d2), 3)} / {round(w_sum(d1), 3)}')

взвешенная цена айтема в тесте: 0.031
потрачено на разметку теста: 81.312
81.312 / 0.031


Выделим, сколько составила средняя часовая ставка для разметки тестовой части датасета. Это будет простое среднее из следующих величин: количество заданий, которое разметчк может сделать за час на основе данного задания, помноженное на цену задания.

In [7]:
def get_hour_pay(df):
    try:
        times = pd.to_datetime(df['ASSIGNMENT:submitted']) - pd.to_datetime(df['ASSIGNMENT:started'])
    except Exception as e:
        times = []
        for i in range(len(assignments_no_control_no_null)):
            try:
                start = pd.to_datetime(assignments_no_control_no_null['ASSIGNMENT:started'].iloc[i])
            except Exception as e:
                start = pd.to_datetime(assignments_no_control_no_null['ASSIGNMENT:started'].apply(lambda x: x.split('T')[1]).iloc[i])
            try:
                end = pd.to_datetime(assignments_no_control_no_null['ASSIGNMENT:submitted'].iloc[i])
            except Exception as e:
                start = pd.to_datetime(assignments_no_control_no_null['ASSIGNMENT:submitted'].apply(lambda x: x.split('T')[1]).iloc[i])
            delta = end - start
            times.extend([delta])
        times = pd.Series(times)
        # times = pd.to_datetime(df['ASSIGNMENT:submitted'].apply(lambda x: x.split('T')[1])) - pd.to_datetime(df['ASSIGNMENT:started'].apply(lambda x: x.split('T')[1]))
    sums = 3600 / times.apply(lambda x: x.seconds) * df['ASSIGNMENT:reward']
    return sums.mean()

get_hour_pay(assignments_no_control_no_null)

2.359464540236515

### Сбор ответов разметчиков и голосование

Собираем ответы голосования большинством для каждого задания.

In [8]:
from collections import defaultdict

text_dict = defaultdict(list)

for task, op1, op2, op3, op4, user, out in zip(
    assignments_no_control_no_null["INPUT:question"], assignments_no_control_no_null["INPUT:option_a"],
    assignments_no_control_no_null["INPUT:option_b"], assignments_no_control_no_null["INPUT:option_c"], 
    assignments_no_control_no_null["INPUT:option_d"],
    assignments_no_control_no_null["ASSIGNMENT:worker_id"], assignments_no_control_no_null["OUTPUT:answer"]
    ):
    if user not in bad_users:
        text_dict[(task, op1, op2, op3, op4)].append([
                user,
                {"out": out}
        ])

print(len(text_dict))

525


In [9]:
keys = list(text_dict.keys())
Counter([len(text_dict[keys[i]]) for i in range(len(keys))])

Counter({5: 415, 4: 99, 3: 10, 2: 1})

Есть 110 заданий, где перекрытие меньше 5. Для формирования итоговых лейблов нужно, чтобы было простое большинство разметчиков, проголосовавших за данную опцию. Если большинства нет, то оценка строится, исходя из оценки навыков разметчиков. В таком случае, финальный лейбл будет присвоен по голосу группы с наилучшими навыками. Если по навыкам будет равенство, то решаем по ответам топ-3 по навыкам разметчиков. Если и данный способ дает равенство, то используются оценки навыков разметчиков из EM-алгоритма (реализация GLAD).

In [10]:
preds_full = {}
user2skill = {k:v for k, v in zip(skills['worker_id'], skills['skill_value'])}
control_acc = assignments[assignments['GOLDEN:answer'].notna()]\
    .groupby('ASSIGNMENT:worker_id')\
        .apply(lambda x: (np.array(x['OUTPUT:answer']) == np.array(x['GOLDEN:answer'])).mean())
user2control = {k:v for k, v in zip(control_acc.index, control_acc.values)}

from crowdkit.aggregation.classification.glad import GLAD

full = assignments['INPUT:question'] + ' ' + assignments['INPUT:option_a'] + ' ' + assignments['INPUT:option_b'] + ' ' + assignments['INPUT:option_c'] + ' ' + assignments['INPUT:option_d']
id2task = dict(enumerate(full))
task2id = {k:v for v, k in id2task.items()}
id2user = dict(enumerate(assignments['ASSIGNMENT:worker_id']))
user2id = {k:v for v, k in id2user.items()}

codes = full.map(task2id)
res = pd.DataFrame({'task': codes, 'worker': assignments['ASSIGNMENT:worker_id'].map(user2id), 'label': assignments['OUTPUT:answer']})
model = GLAD(n_iter=10000, tol=1e-06, m_step_max_iter=1000, m_step_tol=1e-03)
model.fit(res)
user2alpha = dict(enumerate(model.alphas_))
tb = model.alphas_.copy()
tb.index = tb.index.map(id2user)
user2alpha = {k:v for k, v in zip(tb.index, tb.values)}

stats = {
    'total_agreement': 0,
    'majority': 0,
    'skill_based': 0,
    'major_based': 0,
    'em_based': 0,
    'rest': 0,
}

for i in range(len(keys)):
    ans = text_dict[keys[i]]
    lst = [[ans[j][0], ans[j][1]['out']] for j in range(len(ans))]
    users, votes = list(zip(*lst))
    cnt = pd.Series(Counter(votes)).sort_values(ascending=False)

    # # total agreement
    if len(cnt) == 1:
        res = cnt.index[0]
        stats['total_agreement'] += 1
    # simple majority
    elif cnt.iloc[0] > cnt.iloc[1]:
        res = cnt.index[0]
        stats['majority'] += 1
    # (> 1 options) & (1 option == 2 option)
    else:
        # try overall skill based comparison
        vals = list(map(lambda x: user2skill[x], users))
        table = pd.DataFrame({'user': users, 'votes': votes, 'skill': vals})
        agg = table.groupby('votes').agg(
            sum_skill=pd.NamedAgg(column='skill', aggfunc='sum'),
            sum_votes=pd.NamedAgg(column='user', aggfunc='count')
        ).sort_values(by=['sum_votes', 'sum_skill'], ascending=False)
        # check there is a leader by skills
        if agg['sum_skill'].iloc[0] > agg['sum_skill'].iloc[1]:
            res = agg.index[0]
            stats['skill_based'] += 1
        else:
            # top-3 answers by overall skills
            vals = list(map(lambda x: user2skill[x], users))
            table = pd.DataFrame({'user': users, 'votes': votes, 'skill': vals})
            table = table.sort_values(by='skill', ascending=False)
            if len(table) >= 3:
                sub = table.iloc[:3]
            else:
                sub = table
            agg = sub.groupby('votes').agg(
                sum_skill=pd.NamedAgg(column='skill', aggfunc='sum'),
                sum_votes=pd.NamedAgg(column='user', aggfunc='count')
            ).sort_values(by=['sum_votes', 'sum_skill'], ascending=False)
            if agg['sum_skill'].iloc[0] != agg['sum_skill'].iloc[1]:
                res = agg.index[0]
                stats['major_based'] += 1
            
            else:
                vals = list(map(lambda x: user2alpha[x], users))
                table = pd.DataFrame({'user': users, 'votes': votes, 'skill': vals})
                agg = table.groupby('votes').agg(
                    sum_skill=pd.NamedAgg(column='skill', aggfunc='sum'),
                    sum_votes=pd.NamedAgg(column='user', aggfunc='count')
                ).sort_values(by=['sum_votes', 'sum_skill'], ascending=False)
                # check there is a leader by skills
                if agg['sum_skill'].iloc[0] != agg['sum_skill'].iloc[1]:
                    res = agg.index[0]
                    stats['em_based'] += 1
                else:
                    res = agg.index[0]
                    stats['rest'] += 1

    preds_full[keys[i]] = res

In [11]:
stats

{'total_agreement': 356,
 'majority': 155,
 'skill_based': 11,
 'major_based': 2,
 'em_based': 1,
 'rest': 0}

In [12]:
preds_full_df = pd.concat([pd.DataFrame(preds_full.keys(), columns=['task', 'op1', 'op2', 'op3', 'op4']), pd.DataFrame(preds_full.values(), columns=['lb'])], axis=1)

### Сопоставление разметки и ground truth

Забираем задания из датасета с правильными ответами.

In [13]:
res_df = pd.read_csv('general_wa.tsv', sep='\t')

In [14]:
res_df = res_df.rename({
    'INPUT:question': 'task',
    'INPUT:option_a': 'op1',
    'INPUT:option_b': 'op2',
    'INPUT:option_c': 'op3',
    'INPUT:option_d': 'op4',
    'GOLDEN:answer': 'lb',
}, axis=1)

После скачивания с Толоки в текстах рушится форматирование, потому нельзя просто сделать join двух табличек. Нужно убрать все "лишнее" форматирование сразу из двух табличек, чтобы остались только тексты, пунктуация и пробелы.

In [15]:
def format_text(text):
    text = (text.strip().replace('\n', ' ').replace('\t', ' ')
            .replace('\r', ' ').replace('  ', ' ').replace('  ', ' ')
            .replace('  ', ' '))
    return text

res_df['task'] = res_df['task'].apply(format_text)
res_df['op1'] = res_df['op1'].apply(format_text)
res_df['op2'] = res_df['op2'].apply(format_text)
res_df['op3'] = res_df['op3'].apply(format_text)
res_df['op4'] = res_df['op4'].apply(format_text)

preds_full_df['task'] = preds_full_df['task'].apply(format_text)
preds_full_df['op1'] = preds_full_df['op1'].apply(format_text)
preds_full_df['op2'] = preds_full_df['op2'].apply(format_text)
preds_full_df['op3'] = preds_full_df['op3'].apply(format_text)
preds_full_df['op4'] = preds_full_df['op4'].apply(format_text)

res_df['full'] = res_df['task'] + ' ' + res_df['op1'] + ' ' + res_df['op2'] + ' ' + res_df['op3'] + ' ' + res_df['op4']
preds_full_df['full'] = preds_full_df['task'] + ' ' + preds_full_df['op1'] + ' ' + preds_full_df['op2'] + ' ' + preds_full_df['op3'] + ' ' + preds_full_df['op4']

Делаем left join, чтобы соединить голосование и правильные метки для одних и тех же заданий.

In [16]:
new = res_df.merge(preds_full_df.drop(['task', 'op1', 'op2', 'op3', 'op4'], axis=1), on='full', how='left')

In [17]:
new_valid = new[new['lb_y'].notna()].copy()
len(new_valid)

525

Ни одна строка не была утеряна.

In [18]:
new_valid.head(1)

,lb_x,task,op1,op2,op3,op4,full,lb_y
0,A,"Человеческое тело получает энергию, необходиму...",пищи,почвы,солнечного света,воды,"Человеческое тело получает энергию, необходиму...",A


### Подсчет метрики

Если в правом столбце меток осталось 499 непустых строк, значит, форматирование было подчищено корректно и ничего не потерялось

Попробуем посчитать разные метрики

In [19]:
(new_valid['lb_x'] == new_valid['lb_y']).mean()

0.9352380952380952

In [20]:
f1_score(new_valid['lb_x'], new_valid['lb_y'], average='macro')

0.9352531240689137

`Accuracy = 0.935`, `F1_macro = 0.935`